# Advanced OOP
Here, I follow the LinkedIn Learning course [Advanced Python: Object-Oriented
Programming](https://www.linkedin.com/learning/advanced-python-object-oriented-programming/advanced-object-oriented-programming-oop?resume=false&u=72605090)
by Miki Tebeka and try the code. Here is the [course
repo](https://github.com/LinkedInLearning/advanced-python-object-oriented-programming-4510177)
(it is very good). I have added quite some additional explanations from ChatGPT, that I
needed to understand, and also some additional exercises.

## Part 2: Methods

### Using class methods

- mostly used to as a different way to create an instance of a class
    - for instance see the many ways to instantiate pythons [datetime objects](https://docs.python.org/3/library/datetime.html#datetime-objects)

In [ ]:
class Auth:
    def __init__(self, db):
        self.db = db

    def from_token(self, token):
        return self.db.get(token)

# that's our database
auth = Auth({
    'b92d877': 'carly',
    '18317ac': 'elliot',
})


class User:
    def __init__(self, login):
        self.login = login
        # TODO: More fields

    @classmethod # as alternate constructor
    def from_token(cls, token):
        login = auth.from_token(token)
        return cls(login)

class Admin(User):
    ...  # TODO
    

u = User('carly')
print(u.login, type(u))

u = User.from_token('b92d877')
print(u.login, type(u))

a = Admin.from_token('18317ac')
print(a.login, type(a))

carly <class '__main__.User'>
carly <class '__main__.User'>
elliot <class '__main__.Admin'>


### Using static methods

- functions that are assigned to a class
- only reason: discoverability

In [8]:
from random import choice

adjectives = ['cool', 'funny', 'strong']
names = ['bruce', 'carol', 'natasha']


class VM:
    def __init__(self):
        self.name = VM.random_name()


    @staticmethod
    def random_name():
        adjective, name = choice(adjectives), choice(names)
        return f'{adjective}_{name}'


# %% Test
vm = VM()
print(vm.name)

strong_bruce


### Mixin classes

- add functionality to other classes

In [9]:
import logging


logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s: %(message)s',
)

class LoggerMixin:
    def log_id(self):
        logging.info('%s with id %r', self.name, self.id)


class User:
    def __init__(self, name, id):
        self.name = name
        self.id = id


class VM:
    def __init__(self, name, id):
        self.name = name
        self.id = id

class LoggedUser(LoggerMixin, User):
    pass

class LoggedVM(LoggerMixin, VM):
    pass


user = LoggedUser('root', 1)
user.log_id()

vm = LoggedVM('m1', '4922a77')
vm.log_id()

INFO: root with id 1
INFO: m1 with id '4922a77'


### Abstract base classes

- this is to discover errors (here: typo in method name) as early as possible

- using an abstract base class makes the error discoverable in build time rather than in
  test time
    - (I still think this improvement in error discovery is minimal though)

- very practical with mixin classes, because it can be specified what the mixin expects
  from the lass it enhances

In [10]:
from abc import ABC, abstractmethod


class Plugin(ABC):
    @abstractmethod
    def notify(self, event):
        pass

    @abstractmethod
    def shutdown(self):
        pass


class LoggingPlugin(Plugin):
    def notify(self, event):
        print(f'got {event}')

    def shutdown(self):
        print('logger shutting down')


class SecurityPlugin(Plugin):
    def notify(self, event):
        if event.action == 'login' and event.user == 'elliot':
            print(f'WARNING: {event.user} has logged in')

    def shutdwon(self): ######## watch the typo
        print('security shutting down')


def notify(plugins, event):
    for plugin in plugins:
        plugin.notify(event)


def shutdown(plugins):
    for plugin in plugins:
        plugin.shutdown()



class Event:
    def __init__(self, user, action):
        self.user = user
        self.action = action

plugins = [LoggingPlugin(), SecurityPlugin()]
event = Event('elliot', 'login')
notify(plugins, event)
shutdown(plugins)

TypeError: Can't instantiate abstract class SecurityPlugin without an implementation for abstract method 'shutdown'

In [11]:
# the mixin class specifies what methods are expected in the class it enhances

from abc import ABC, abstractmethod

class SaveMixin(ABC):
    @abstractmethod
    def serialize(self):
        pass

    def save(self, filepath):
        data = self.serialize()
        with open(filepath, 'w') as f:
            f.write(data)


### Defining interfaces with `typing.Protocol`

In [ ]:
from typing import Protocol


class Writer(Protocol):
    # that's only for defining the signatures of the functions and the expected types
    def write(self, data: bytes) -> None:
        ... # code doesn't matter


import json

def store_json(w: Writer, obj: dict) -> None:
    # expects an object adhering to the Writer protocol passed as `w`
    data = json.dumps(obj).encode('utf-8') # sends bytes to the object passed as `w`
    w.write(data)


class S3File:
    # does not inherit from Writer, but would be expected to adher to it
    def write(self, data: str) -> None: # type mismatch! expects str, but gets bytes; would raise when running mypy
        print(f's3: write: {data!r}')


out = S3File()
obj = {
    'id': '007',
    'lat': 51.4871871,
    'lng': -0.1270605,
}
store_json(out, obj) 
# `store_json` gets passed an object not adhering to the Writer protocol, but still
# treats it as if it was, printing the data in the write method as bytes (which shows by
# the `b'` in front of the dict: it’s treating the bytes as a string).

s3: write: b'{"id": "007", "lat": 51.4871871, "lng": -0.1270605}'


## Part 3: Special Methods
tbc